In [1]:
import numpy as np
import pandas as pd

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [8]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [9]:
data = pd.read_csv("Real_Combine.csv")
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [16]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

In [20]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int("num layers",2,30)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value = 150,
                                            max_value = 400,
                                            step = 32 ),
                               activation = 'relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model    
                               

In [22]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='projects',
    project_name='Air Quality Index')

In [23]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 30, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 150, 'max_value': 400, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 150, 'max_value': 400, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [26]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: 52.833413441975914

Best val_mean_absolute_error So Far: 50.38274129231771
Total elapsed time: 00h 01m 15s
INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.results_summary()

Results summary
Results in projects\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num layers: 16
units_0: 278
units_1: 150
learning_rate: 0.001
units_2: 246
units_3: 246
units_4: 246
units_5: 278
units_6: 278
units_7: 246
units_8: 374
units_9: 342
units_10: 182
units_11: 278
units_12: 214
units_13: 214
units_14: 278
units_15: 214
units_16: 278
units_17: 310
units_18: 374
units_19: 310
units_20: 246
Score: 50.38274129231771
Trial summary
Hyperparameters:
num layers: 12
units_0: 182
units_1: 310
learning_rate: 0.001
units_2: 214
units_3: 246
units_4: 374
units_5: 278
units_6: 214
units_7: 246
units_8: 182
units_9: 310
units_10: 214
units_11: 374
units_12: 310
units_13: 214
units_14: 214
units_15: 278
units_16: 342
units_17: 214
units_18: 310
units_19: 278
units_20: 374
units_21: 246
units_22: 150
units_23: 310
units_24: 374
units_25: 342
units_26: 374
units_27: 374
Score: 52.833413441975914
Trial summary